## 임베딩 Embedding
자연어를 숫자나 벡터 형태로 변환할 필요가 있다.<br>
단어나 문장을 수치화해 벡터 공간으로 표현하는 과정이다.<br>

종류에는 단어 임베딩과 문장 임베딩이 있다.<br>

### 단어 임베딩 - 원-핫 인코딩
단어를 숫자 벡터로 변환하는 가장 기본적인 방법이다. <br>
단 하나의 값만 1이고 나머지 요솟값은 0인 인코딩이다.<br>
결과를 원-핫 벡터라고 한다.<br>

In [4]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 리스트형태로 명사를 추출 
nouns = komoran.nouns(text)
print(nouns) 


# 딕셔너리형태로 단어 사전 구축 및 단어별 인덱스 부여, 중복 제외
dics = {}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)

# 원-핫 인코딩
nb_classes = len(dics) # 원-핫 벡터 크기 결정
targets = list(dics.values()) # 단어들을 리스트로 변환
one_hot_targets = np.eye(nb_classes)[targets] # 단위행렬 반환
print(one_hot_targets)

['오늘', '날씨', '구름']
{'오늘': 0, '날씨': 1, '구름': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


### Word2Vec
신경망 기반 단어 인베딩의 대표적인 방법<br>
원-핫 인코딩은 단어 간 유사도 계산에서 좋은 성능을 보이지 않는다.<br>

Word2Vec 모델의 제안
- CBOW : 맥락이라 표현되는 주변 단어들을 이용해 타깃 단어로 설정해 학습된 가중치 데이터를 임베딩 벡터로 활용
- skip-gram : 하나의 타깃 단어를 이용해 주변 단어들을 예측하는 신경망 모델

파라미터 종류
- sentences : 문장 데이터
- vector_size : 단어 임베딩 벡터의 차원
- window : 주변 단어 윈도우의 크기
- hs : 1(softmax 사용), 0(음수 샘플링 사용)
- min_count : 단어 최소 빈도 수 제한
- sg : 0(CBOW 모델), 1(skip-gram 모델)

In [16]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

# 네이버 영화 리뷰 데이터 읽어오기
def read_review_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

# 학습 시간 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./nsmc/ratings.txt')
print(len(review_data))
print('~ 말뭉치 데이터 읽기 완료 : ', time.time()-start)

# 문장 단위로 명사만 추출해 학습 데이터로 만들기
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('~ 형태소에서 명사만 추출 완료 : ', time.time() - start)

# Word2Vec 모델 학습
print('3) Word2Vec 모델 학습')
model = Word2Vec(sentences = docs, vector_size=200, window=4, hs=1, min_count=2, sg=1)
print('~ Word2Vec 모델 학습 완료 : ', time.time() - start)

# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('~ 학습된 모델 저장 완료 : ', time.time() - start)

# 학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print('corpus_count : ', model.corpus_count)
print('corpus_total_words : ', model.corpus_total_words)

1) 말뭉치 데이터 읽기 시작
200000
~ 말뭉치 데이터 읽기 완료 :  0.3729889392852783
2) 형태소에서 명사만 추출 시작
~ 형태소에서 명사만 추출 완료 :  72.13897013664246
3) Word2Vec 모델 학습
~ Word2Vec 모델 학습 완료 :  86.99039793014526
4) 학습된 모델 저장 시작
~ 학습된 모델 저장 완료 :  87.35029196739197
corpus_count :  200000
corpus_total_words :  1076896


#### 단어 임베딩된 값과 벡터 공간상의 유사한 단어 확인

In [19]:
from gensim.models import Word2Vec

# 모델 로딩
# model = Word2Vec('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))

corpus_total_words :  1076896
사랑 :  [-0.06599452 -0.25043118  0.03878808 -0.10320264 -0.03503748 -0.49748486
 -0.0604511   0.34850934  0.16055478  0.16367863 -0.04023429 -0.27490973
  0.07520173  0.25263584 -0.24258521  0.10076682  0.029464   -0.03069105
 -0.10789102 -0.4216866  -0.03179588 -0.00557936 -0.23307727  0.12833916
 -0.21739204 -0.18563803  0.18058497 -0.42408815 -0.19688345  0.00119747
  0.12856676 -0.00519366 -0.14584608 -0.48853344  0.14520757  0.29185116
 -0.096228   -0.2767676  -0.13547686 -0.08943529 -0.10968708  0.11850785
 -0.18150565 -0.09706109  0.19900104  0.1423771  -0.22967462 -0.07249592
  0.448221   -0.01817471  0.05876061 -0.1080583   0.01785472  0.08719216
  0.41917527  0.09034201  0.38950205 -0.22915351 -0.17946567 -0.22628713
 -0.11323027 -0.08122098 -0.36301428  0.06868676  0.02999543 -0.03827934
 -0.50370127  0.18922721  0.04170812  0.37191242  0.01723982 -0.28787655
  0.09012057 -0.05315329  0.18349749 -0.07182185  0.22727905 -0.03927505
 -0.27726147  0

>자연어 그대로를 처리할 수 없으므로 연산 가능한 형태인 벡터 형태로 변환하는 것을 임베딩이라고 한다.<br>
말뭉치의 품질과 데이터양이 충분하다면 훌륭한 품질의 임베딩 모델을 구축할 수 있다.<br>
이런 임베딩은 일반적으로 신경망 모델의 입력으로 많이 사용된다.<br>